In [1]:
library(ppcor)
#install.packages('lmPerm')
library(lmPerm)
library(psych)
library(car)
library(ggplot2)
library(lme4)
library(mediation)
library(caret)
library(gbm)
library(party)
library(Metrics)
#install.packages("wesanderson")
library(wesanderson)
library(gtools)
library(interactions)
#install.packages('ggstance')
library(ggstance)
#install.packages('elasticnet')
library(elasticnet)
#install.packages('kernlab')
library(kernlab)
#install.packages('e1071')
library(e1071)
#install.packages('MatchIt')
library(MatchIt)
library(dplyr)
library(ukbtools)
library(mediation)

In [2]:
my_ukb_data <- ukb_df("ukb38276", path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
load("/home/bic/fmorys/Projects/UKBB_all/di.Rda")
colnames(di)=c('eid',colnames(di)[2:length(di)])

ukbb_check=merge(my_ukb_data,di, by.x='eid')

di$bipolar_dis <- NA
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type I (Mania)"] <- 1
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type II (Hypomania)"] <- 1

#Personality Disorders
di$personality_dis <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$personality_dis[di[i]=="A personality disorder"] <- 1
}

#Schizophrenia
di$schizophrenia <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$schizophrenia[di[i]=="Schizophrenia"] <- 1
}


# Exclude participants who had a heart attack, angina or stroke
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('vascularheart_problems_diagnosed_by_doctor', 
                          colnames(di)))) { #Columns with vascular diseases
di$vascular_heart_diagnoses[di[i]=="Heart attack"] <-1
di$vascular_heart_diagnoses[di[i] =="Angina"] <- 1
di$vascular_heart_diagnoses[di[i] =="Stroke"] <- 1
}


#Dementia
for (i in as.numeric(grep('noncancer_illness_code_selfreported', 
                          colnames(di)))) { #Columns with noncancer illness for all timepoints
di$neurological_disorder[di[i] == "1263"] <- 1 #Dementia or Alzheimer's
di$neurological_disorder[di[i] == "1262"] <- 1 #Parkinson Disease
di$neurological_disorder[di[i] == "1258"] <- 1 #Chronic degenerative neurological
di$neurological_disorder[di[i] == "1256"] <- 1 #Guillain Barré
di$neurological_disorder[di[i] == "1261"] <- 1 #Multiple slerosis
di$neurological_disorder[di[i] == "1297"] <- 1 #Other demyenilating disease
di$neurological_disorder[di[i] == "1081"] <- 1 #Stroke
di$neurological_disorder[di[i] == "1032"] <- 1 #Brain cancer
di$neurological_disorder[di[i] == "1491"] <- 1 #Brain hemorrhage
di$neurological_disorder[di[i] == "1245"] <- 1 #Brain/ intracranial abcess
di$neurological_disorder[di[i] == "1425"] <- 1 #Cerebral aneurism
di$neurological_disorder[di[i] == "1433"] <- 1 #Cerebral palsy
di$neurological_disorder[di[i] == "1256"] <- 1 #Encephalitis
di$neurological_disorder[di[i] == "1264"] <- 1 #Epilepsy
di$neurological_disorder[di[i] == "1266"] <- 1 #Head injury
di$neurological_disorder[di[i] == "1244"] <- 1 #Infections of the nervous system
di$neurological_disorder[di[i] == "1583"] <- 1 #Ischaemic stroke
di$neurological_disorder[di[i] == "1031"] <- 1 #Meningeal cancer
di$neurological_disorder[di[i] == "1659"] <- 1 #Meningioma (benign)
di$neurological_disorder[di[i] == "1247"] <- 1 #Meningitis
di$neurological_disorder[di[i] == "1259"] <- 1 #Motor neuron disease
di$neurological_disorder[di[i] == "1240"] <- 1 #Neurological injury/trauma
di$neurological_disorder[di[i] == "1524"] <- 1 #Spina bifida
di$neurological_disorder[di[i] == "1083"] <- 1 #Subdural hematoma
di$neurological_disorder[di[i] == "1086"] <- 1 #Subarachnoid haemorrage
di$neurological_disorder[di[i] == "1082"] <- 1 #Transient ischaemic attack
}

#If BMI < 18.5 (Re-do this, cause it's not usually done)
#di$underweight1[di$body_mass_index_bmi_f21001_0_0 < 18.5] <- 1
#di$underweight2[di$body_mass_index_bmi_f21001_1_0 < 18.5] <- 1
di$underweight3[di$body_mass_index_bmi_f21001_2_0 < 18.5] <- 1

#Exclude participants if no neuroimaging data, bipolar_dis == 1; personality_dis == 1; schizophrenia == 1; vascular_heart_diagnoses ==1

di$excluded <- NA
di$excluded[di$bipolar_dis ==1] <- 1
di$excluded[di$personality_dis == 1] <- 1
di$excluded[di$schizophrenia == 1] <- 1
di$excluded[di$vascular_heart_diagnoses ==1] <- 1
#di$excluded[di$underweight1 ==1] <- 1
#di$excluded[di$underweight2 ==1] <- 1
di$excluded[di$underweight3 == 1] <- 1
di$excluded[di$neurological_disorder ==1] <- 1

di$included<-car::recode(di$excluded, "1='excluded'; else='included'")


#Select subjects if included ==1
di_excluded <- subset(di, included=="included")

ukbb_all=merge(my_ukb_data, di_excluded, by.x='eid')

nrow(ukbb_all)

variables_for_outliers=c(58:88, 120:150, 26:27, 32:38, 42:43, 48:54, 89:119, 151:181, 1891, 9) # all volume and thickness variables from FS, BMI, eTIV 

for (j in variables_for_outliers) { # 
    Q3=as.numeric(quantile(ukbb_all[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(ukbb_all[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    ukbb_all[j][ukbb_all[j]<lower]=NA
    ukbb_all[j][ukbb_all[j]>upper]=NA
}

ukbb_all=ukbb_all[complete.cases(ukbb_all[,variables_for_outliers]),]
nrow(ukbb_all)

ukbb_all$WMH_logscaled=log(ukbb_all$total_volume_of_white_matter_hyperintensities_from_t1_and_t2_flair_images_f25781_2_0*
                    ukbb_all$volumetric_scaling_from_t1_head_image_to_standard_space_f25000_2_0)
ukbb_all$logBMI3=log(ukbb_all$body_mass_index_bmi_f21001_2_0)

## BMI -> WMH -> brain measures

# BMI -> WMH - stays the same for all models

In [3]:
set.seed(2014)
med.fit <- lm(scale(ukbb_all$WMH_logscaled) ~
          scale(ukbb_all$logBMI3) +
          scale(ukbb_all$age_when_attended_assessment_centre_f21003_2_0) +
          ukbb_all$sex_f31_0_0, data=ukbb_all)

## WMH -> CT

In [4]:
Stats=matrix(nrow=length(c(58:88, 120:150)),ncol=10) # CT variables
index=1

for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
  
    out.fit <- lm(scale(ukbb_all[[i]]) ~ 
                scale(ukbb_all$WMH_logscaled) +
                scale(ukbb_all$logBMI3) +
                scale(ukbb_all$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_all$sex_f31_0_0, data=ukbb_all)
    
    med.out <- mediate(med.fit, out.fit, treat='scale(ukbb_all$logBMI3)',
                   mediator='scale(ukbb_all$WMH_logscaled)',
                   robustSE=TRUE,
                   sims=1000, data=ukbb_all)
    
    med2dep <- summary(lm(scale(ukbb_all[[i]]) ~ 
                scale(ukbb_all$WMH_logscaled) +
                scale(ukbb_all$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_all$sex_f31_0_0, data=ukbb_all)) 
      
    
    Stats[index,][1]=colnames(ukbb_all)[i]
    Stats[index,][2]=med.out$d0 # ACME estimate
    Stats[index,][3]=med.out$d0.p # ACME p-value
    Stats[index,][4]=med.out$z0 # ADE e
    Stats[index,][5]=med.out$z0.p # ADE p
    Stats[index,][6]=med.out$tau.coef # Total e
    Stats[index,][7]=med.out$tau.p # Total p
    Stats[index,][8]=med.out$n0 # Proportion mediated e
    Stats[index,][9]=med.out$n0.p # Proportion mediated p
    Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> CT
       
     
    index=index+1
}
Stats_adj=Stats
Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

In [6]:
write.table(Stats_adj[,2:10],'/dagher/dagher11/filip/OBAL/output/WMH_CT_mediation.csv',sep=',',quote=FALSE, row.names=FALSE)

## WMH -> cortical volume

In [ ]:
Stats=matrix(nrow=length(c(89:119, 151:181)),ncol=10) # volume variables
index=1

for (i in c(89:119, 151:181)) { #All volume variables 
  
    out.fit <- lm(scale(ukbb_all[[i]]) ~ 
                scale(ukbb_all$WMH_logscaled) +
                scale(ukbb_all$logBMI3) +
                scale(ukbb_all$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_all$sex_f31_0_0, data=ukbb_all)
    
    med.out <- mediate(med.fit, out.fit, treat='scale(ukbb_all$logBMI3)',
                   mediator='scale(ukbb_all$WMH_logscaled)',
                   robustSE=TRUE,
                   sims=1000, data=ukbb_all)

    med2dep <- summary(lm(scale(ukbb_all[[i]]) ~ 
                scale(ukbb_all$WMH_logscaled) +
                scale(ukbb_all$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_all$sex_f31_0_0, data=ukbb_all)) 
      
    
    Stats[index,][1]=colnames(ukbb_all)[i]
    Stats[index,][2]=med.out$d0 # ACME estimate
    Stats[index,][3]=med.out$d0.p # ACME p-value
    Stats[index,][4]=med.out$z0 # ADE e
    Stats[index,][5]=med.out$z0.p # ADE p
    Stats[index,][6]=med.out$tau.coef # Total e
    Stats[index,][7]=med.out$tau.p # Total p
    Stats[index,][8]=med.out$n0 # Proportion mediated e
    Stats[index,][9]=med.out$n0.p # Proportion mediated p
    Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol
       
     
    index=index+1
    
}

Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

In [ ]:
write.table(Stats_adj[,2:9],'/dagher/dagher11/filip/OBAL/output/WMH_VOL_mediation.csv',sep=',',quote=FALSE, row.names=FALSE)